In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    auc,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

import joblib
import json
import os

In [10]:
STOCKS = ["CGH", "LICN", "NABIL", "NIFRA", "UPPER"]
DATA_DIR = "data"
MODEL_DIR = "models"
METRIC_DIR = "metrics"

os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(METRIC_DIR, exist_ok=True)


In [11]:
def create_sequences(data, window=30):
    X, y = [], []
    for i in range(window, len(data)):
        X.append(data[i-window:i])
        y.append(data[i])
    return np.array(X), np.array(y)


def clean_close_series(df):
    if "close" not in df.columns:
        raise ValueError("Expected a 'close' column in the dataset")
    close = pd.to_numeric(df["close"], errors="coerce")
    close = close.interpolate(method="linear").ffill().bfill()
    close = close.dropna()
    return close.values.reshape(-1, 1)


def split_time_series(X, y, train_ratio=0.7, val_ratio=0.15):
    n = len(X)
    train_end = int(n * train_ratio)
    val_end = int(n * (train_ratio + val_ratio))
    X_train = X[:train_end]
    y_train = y[:train_end]
    X_val = X[train_end:val_end]
    y_val = y[train_end:val_end]
    X_test = X[val_end:]
    y_test = y[val_end:]
    return X_train, y_train, X_val, y_val, X_test, y_test

In [12]:
for stock in STOCKS:
    print(f"\nTraining LSTM for {stock}")

    df = pd.read_csv(f"{DATA_DIR}/{stock}.csv")
    close_prices = clean_close_series(df)

    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(close_prices)

    X, y = create_sequences(scaled)
    X = X.reshape((X.shape[0], X.shape[1], 1))

    X_train, y_train, X_val, y_val, X_test, y_test = split_time_series(X, y)

    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
        Dropout(0.2),
        LSTM(50),
        Dropout(0.2),
        Dense(1)
    ])

    model.compile(optimizer="adam", loss="mse")
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=20,
        batch_size=32,
        verbose=0
    )

    model.save(f"{MODEL_DIR}/{stock}_lstm.h5")
    joblib.dump(scaler, f"{MODEL_DIR}/{stock}_scaler.pkl")

    history_payload = {
        "loss": [float(v) for v in history.history.get("loss", [])],
        "val_loss": [float(v) for v in history.history.get("val_loss", [])],
        "final_loss": float(history.history["loss"][-1]),
        "final_val_loss": float(history.history["val_loss"][-1]),
        "generalization_gap": float(history.history["val_loss"][-1] - history.history["loss"][-1])
    }
    with open(f"{METRIC_DIR}/{stock}_loss.json", "w") as f:
        json.dump(history_payload, f)

    # Save regression curve + error trend on test split
    preds = model.predict(X_test)
    actual = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    predicted = scaler.inverse_transform(preds).flatten()
    errors = actual - predicted

    pd.DataFrame({
        "actual": actual.tolist(),
        "predicted": predicted.tolist()
    }).to_json(f"{METRIC_DIR}/{stock}_regression_curve.json")

    pd.DataFrame({
        "actual": actual.tolist(),
        "predicted": predicted.tolist(),
        "error": errors.tolist()
    }).to_json(f"{METRIC_DIR}/{stock}_error_trend.json")

    regression_metrics = {
        "mse": float(mean_squared_error(actual, predicted)),
        "rmse": float(np.sqrt(mean_squared_error(actual, predicted))),
        "mae": float(mean_absolute_error(actual, predicted)),
        "r2": float(r2_score(actual, predicted))
    }
    with open(f"{METRIC_DIR}/{stock}_regression_metrics.json", "w") as f:
        json.dump(regression_metrics, f)


Training LSTM for CGH


c:\Users\samaw\Desktop\project\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step

Training LSTM for LICN


c:\Users\samaw\Desktop\project\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step

Training LSTM for NABIL


...

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step

Training LSTM for NIFRA


c:\Users\samaw\Desktop\project\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step

Training LSTM for UPPER


c:\Users\samaw\Desktop\project\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step


In [13]:
from tensorflow.keras.models import load_model

FORECAST_DAYS = 30

for stock in STOCKS:
    df = pd.read_csv(f"{DATA_DIR}/{stock}.csv")
    close = df["close"].values.reshape(-1, 1)

    scaler = joblib.load(f"{MODEL_DIR}/{stock}_scaler.pkl")
    model = load_model(f"{MODEL_DIR}/{stock}_lstm.h5", compile=False)

    scaled = scaler.transform(close)
    window = scaled[-60:].reshape(1, 60, 1)

    preds = []
    for _ in range(FORECAST_DAYS):
        next_scaled = model.predict(window, verbose=0)[0][0]
        preds.append(next_scaled)
        next_step = np.array(next_scaled).reshape(1, 1, 1)
        window = np.concatenate([window[:, 1:, :], next_step], axis=1)

    preds_inv = scaler.inverse_transform(np.array(preds).reshape(-1, 1)).flatten()
    out = pd.DataFrame({
        "day": list(range(1, FORECAST_DAYS + 1)),
        "predicted_price": preds_inv
    })
    out.to_csv(f"{MODEL_DIR}/{stock}_30day_forecast.csv", index=False)

print("Saved 30-day forecasts to models/")

Saved 30-day forecasts to models/


In [14]:
def create_labels(prices):
    return (prices[1:] > prices[:-1]).astype(int)


In [15]:
for stock in STOCKS:
    print(f"\nTraining classifier for {stock}")

    df = pd.read_csv(f"{DATA_DIR}/{stock}.csv")
    prices = clean_close_series(df).flatten()

    X = prices[:-1].reshape(-1, 1)
    y = create_labels(prices)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, shuffle=False
    )

    clf = RandomForestClassifier(
        n_estimators=200,
        max_depth=5,
        random_state=42
    )
    clf.fit(X_train, y_train)

    joblib.dump(clf, f"{MODEL_DIR}/{stock}_rf.pkl")

    # Predictions
    probs = clf.predict_proba(X_test)[:, 1]
    preds = clf.predict(X_test)

    # Confusion Matrix
    cm = confusion_matrix(y_test, preds)
    with open(f"{METRIC_DIR}/{stock}_confusion.json", "w") as f:
        json.dump({
            "matrix": cm.tolist(),
            "labels": ["DOWN", "UP"]
        }, f)

    # ROC
    fpr, tpr, _ = roc_curve(y_test, probs)
    roc_auc = auc(fpr, tpr)

    with open(f"{METRIC_DIR}/{stock}_roc.json", "w") as f:
        json.dump({
            "fpr": fpr.tolist(),
            "tpr": tpr.tolist(),
            "auc": float(roc_auc)
        }, f)

    # Classification metrics
    metrics_payload = {
        "accuracy": float(accuracy_score(y_test, preds)),
        "precision": float(precision_score(y_test, preds, zero_division=0)),
        "recall": float(recall_score(y_test, preds, zero_division=0)),
        "f1": float(f1_score(y_test, preds, zero_division=0))
    }
    with open(f"{METRIC_DIR}/{stock}_classification_metrics.json", "w") as f:
        json.dump(metrics_payload, f)

    report = classification_report(y_test, preds, target_names=["DOWN", "UP"], output_dict=True, zero_division=0)
    with open(f"{METRIC_DIR}/{stock}_classification_report.json", "w") as f:
        json.dump(report, f)

    print(f"AUC for {stock}: {roc_auc:.3f}")


Training classifier for CGH
AUC for CGH: 0.502

Training classifier for LICN
AUC for LICN: 0.481

Training classifier for NABIL
AUC for NABIL: 0.499

Training classifier for NIFRA
AUC for NIFRA: 0.426

Training classifier for UPPER
AUC for UPPER: 0.421


In [16]:
rows = []

for stock in STOCKS:
    data = pd.read_json(f"{METRIC_DIR}/{stock}_regression_curve.json")

    mse = mean_squared_error(data["actual"], data["predicted"])
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(data["actual"], data["predicted"])
    r2 = r2_score(data["actual"], data["predicted"])

    rows.append({
        "Stock": stock,
        "MSE": mse,
        "RMSE": rmse,
        "MAE": mae,
        "R2": r2
    })

lstm_table = pd.DataFrame(rows)
lstm_table


,Stock,MSE,RMSE,MAE,R2
0,CGH,4341.629708,65.891044,54.355796,0.143552
1,LICN,8053.173824,89.739478,45.737159,0.944712
2,NABIL,244.344591,15.631526,12.036249,0.886421
3,NIFRA,65.715811,8.106529,5.859517,0.502378
4,UPPER,20.323639,4.508175,3.164738,0.854404
